In [1]:
import pandas as pd
import unicodedata
import re
from difflib import get_close_matches

In [2]:
stats_2025=pd.read_csv('../batter_record_1_hit/stats/batter_stats_2025.csv', encoding='utf-8', 
            encoding_errors='replace')  # replaces invalid bytes with �


In [3]:
stats_2024 = pd.read_csv('../batter_record_1_hit/stats/batter_stats_2024.csv')
# stats_2025 = pd.read_csv('../batter_record_1_hit/stats/batter_stats_2025.csv')
def_rank_2024 = pd.read_csv('def_rank_era_pitchers_2024.csv')
def_rank_2025=pd.read_csv('../scrapers/def_scraper/batter_rbi_def_rank.csv')
batter_team_2024 = pd.read_csv('../batter_record_1_hit/stats/batters_team_2024.csv')
batter_team_2025 = pd.read_csv('../batter_record_1_hit/stats/batters_team_2025.csv')

In [4]:
batter_team_2024['team'].unique()

array(['BOS', 'LAD', 'BAL', 'TEX', 'NYY', 'KCR', 'TOR', 'CIN', 'NYM',
       'COL', 'PIT', 'PHI', 'MIL', 'ATL', 'ARI', 'HOU', 'CLE', '2TM',
       'MIA', 'SDP', 'LAA', 'CHC', 'STL', 'Team', 'TBR', 'SEA', 'SFG',
       'OAK', 'MIN', 'CHW', 'WSN', 'DET', '3TM'], dtype=object)

In [5]:
team_mapping_5 = {
    'CHC': 'CHC',
    'LAD': 'LAD',
    'ARI': 'ARI',
    'PIT': 'PIT',
    'MIA': 'MIA',
    'TOR': 'TOR',
    'BOS': 'BOS',
    'OAK': 'OAK',
    'BAL': 'BAL',
    'SEA': 'SEA',
    'SDP': 'SDP',
    'ATL': 'ATL',
    'PHI': 'PHI',
    'TEX': 'TEX',
    'MIL': 'MIL',
    'WSN': 'WSH',
    'NYY': 'NYY',
    'STL': 'STL',
    'TBR': 'TBR',
    'DET': 'DET',
    'KCR': 'KCR',
    'CLE': 'CLE',
    'COL': 'COL',
    'NYM': 'NYM',
    'CIN': 'CIN',
    'MIN': 'MIN',
    'SFG': 'SFG',
    'HOU': 'HOU',
    'CHW': 'CHW',
    'LAA': 'LAA'
}

batter_team_2025['team'] = batter_team_2025['team'].map(team_mapping_5)
batter_team_2024['team'] = batter_team_2024['team'].map(team_mapping_5)

In [6]:
batter_team_2025['team'].unique()

array(['CHC', 'LAD', 'ARI', 'PIT', 'MIA', 'TOR', 'BOS', nan, 'BAL', 'SEA',
       'SDP', 'ATL', 'PHI', 'TEX', 'MIL', 'WSH', 'NYY', 'STL', 'TBR',
       'DET', 'KCR', 'CLE', 'COL', 'NYM', 'CIN', 'MIN', 'SFG', 'HOU',
       'CHW', 'LAA'], dtype=object)

In [7]:
batter_team_2024[batter_team_2024['player'] == 'Shea Langeliers']

,player,team
133,Shea Langeliers,OAK


In [8]:
def_rank_2025.rename(columns={'Team': 'team'}, inplace=True)

In [9]:
#fixing batter_team_2024
batter_team_2024['team'] = batter_team_2024['team'].replace({'WSN': 'WSH'})
batter_team_2025['team'] = batter_team_2025['team'].replace({'WSN': 'WSH'})

In [10]:
batter_team_2024

,player,team
0,Jarren Duran*,BOS
1,Shohei Ohtani*,LAD
2,Gunnar Henderson*,BAL
3,Marcus Semien,TEX
4,Juan Soto*,NYY
...,...,...
915,Justin Wilson*,CIN
916,Josh Winckowski,BOS
917,Kirby Yates*,TEX
918,Alex Young*,CIN


In [11]:
batter_team_2024[batter_team_2024['player'] == 'Shea Langeliers']

,player,team
133,Shea Langeliers,OAK


In [12]:
stats_2024=stats_2024[['away_hitters','away_RBI','home_hitters','home_RBI']]
stats_2025=stats_2025[['away_hitters','away_RBI','home_hitters','home_RBI']]

In [13]:
#dropping first half of the 2024 season
stats_2024 = stats_2024.iloc[len(stats_2024)//2:]


In [14]:
#normalizing the names, getting rid of accents
def normalize_name(name):
    if not isinstance(name, str):
        return name
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')

In [15]:
batter_team_2024['player'] = batter_team_2024['player'].apply(normalize_name)
batter_team_2025['player'] = batter_team_2025['player'].apply(normalize_name)

In [16]:
def clean_name_symbols(name):
    if not isinstance(name, str):
        return name
    return name.replace('*', '').replace('#', '')

In [17]:
#applying it to the player collumn
batter_team_2024['player'] = batter_team_2024['player'].apply(clean_name_symbols)
batter_team_2025['player'] = batter_team_2025['player'].apply(clean_name_symbols)

In [18]:
# dropping other teams the same player played for, bc they got traded, 2TM stands for 2 teams etc....
# rows_to_drop = set()

# # Loop through the DataFrame
# for i, val in batter_team_2024['team'].items():
#     if val == '2TM':
#         rows_to_drop.update(range(i, i+2))  # this row + 1 more
#     elif val == '3TM':
#         rows_to_drop.update(range(i, i+3))  # this row + 2 more
#     elif val == '4TM':
#         rows_to_drop.update(range(i, i+4))  # this row + 3 more
#     elif val == '5TM':
#         rows_to_drop.update(range(i, i+5))  # this row + 3 more

# # Drop rows
# batter_team_2024 = batter_team_2024.drop(rows_to_drop).reset_index(drop=True)
# Step 1: Remove all rows with '2TM', '3TM', etc. in the 'team' column
multi_team_flags = ['2TM', '3TM', '4TM', '5TM']
filtered_df = batter_team_2024[~batter_team_2024['team'].isin(multi_team_flags)].copy()
filtered_df_2 = batter_team_2025[~batter_team_2025['team'].isin(multi_team_flags)].copy()


# Step 2: Keep only the LAST team listed for each player (i.e., current team)
batter_team_2024 = filtered_df.drop_duplicates(subset='player', keep='last').reset_index(drop=True)
batter_team_2025 = filtered_df_2.drop_duplicates(subset='player', keep='last').reset_index(drop=True)



In [19]:
# # dropping other teams the same player played for, bc they got traded, 2TM stands for 2 teams etc....
# rows_to_drop = set()

# # Loop through the DataFrame
# for i, val in batter_team_2024['team'].items():
#     if val == '2TM':
#         rows_to_drop.update(range(i, i+2))  # this row + 1 more
#     elif val == '3TM':
#         rows_to_drop.update(range(i, i+3))  # this row + 2 more
#     elif val == '4TM':
#         rows_to_drop.update(range(i, i+4))  # this row + 3 more
#     elif val == '5TM':
#         rows_to_drop.update(range(i, i+5))  # this row + 3 more
        
# batter_team_2025 = batter_team_2025.drop(rows_to_drop).reset_index(drop=True)

In [20]:
#matching the batter_team_2024 with the broken names in stats_2024 
# Step 1: Safely extract short names like "P. Sandoval"
def clean_name(raw):
    if not isinstance(raw, str):
        return None

    # Normalize unicode to remove accents (e.g., Hernández → Hernandez)
    raw = unicodedata.normalize('NFKD', raw).encode('ascii', 'ignore').decode('utf-8')

    # Remove leading garbage (e.g., bT. Taylor → T. Taylor)
    raw = re.sub(r"^[^A-Z]*", "", raw)

    # Fix missing space after period (e.g., T.Taylor → T. Taylor)
    raw = re.sub(r"([A-Z])\.([A-Z])", r"\1. \2", raw)

    # Remove trailing position labels (e.g., PH-CF, 3B, DH)
    raw = re.sub(r"[A-Z]{1,3}(-[A-Z]{1,3})?$", "", raw.strip())

    # Remove anything else weird
    raw = re.sub(r"[^\w\s\.\'\-]", "", raw)

    # Match short name: first initial + last name (including compound last names)
    match = re.match(r"([A-Z]\.\s(?:[A-Z][a-z]+(?:\s|['’\-]))*[A-Z][a-z]+)", raw)
    return match.group(1).strip() if match else None

stats_2024['short_name'] = stats_2024['away_hitters'].apply(clean_name)

# Step 2: Match short name to full name in pitchers_team
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # first initial match
            return name

    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None

# Step 3: Apply the matching
full_name_list = batter_team_2024['player'].tolist()
stats_2024['matched_name'] = stats_2024['short_name'].apply(lambda x: match_full_name(x, full_name_list))
def clean_string_column(col):
    return (
        col.astype(str)
           .str.encode('ascii', errors='ignore')
           .str.decode('utf-8')
           .str.replace(r'\s+', ' ', regex=True)
           .str.strip()
    )

# Clean both columns
stats_2024['matched_name'] = clean_string_column(stats_2024['matched_name'])
batter_team_2024['player'] = clean_string_column(batter_team_2024['player'])

# Step 4: Merge the DataFrames on the matched name
merged_df = stats_2024.merge(batter_team_2024, left_on='matched_name', right_on='player', how='left')

In [21]:
batter_team_2024[batter_team_2024['player'] == 'Shea Langeliers']


,player,team
110,Shea Langeliers,OAK


In [22]:
batter_team_2024

,player,team
0,Jarren Duran,BOS
1,Shohei Ohtani,LAD
2,Gunnar Henderson,BAL
3,Marcus Semien,TEX
4,Juan Soto,NYY
...,...,...
737,Justin Wilson,CIN
738,Josh Winckowski,BOS
739,Kirby Yates,TEX
740,Alex Young,CIN


In [23]:
merged_df

,away_hitters,away_RBI,home_hitters,home_RBI,short_name,matched_name,player,team
0,S. LangeliersC,0,C. Walker1B,0,S. Langeliers,Shea Langeliers,Shea Langeliers,OAK
1,T. Soderstrom1B,0,R. GrichukRF,0,T. Soderstrom,Tyler Soderstrom,Tyler Soderstrom,OAK
2,D. CameronRF,0,aC. CarrollPH-CF,0,D. Cameron,Daz Cameron,Daz Cameron,OAK
3,A. Alvarez3B,0,E. Suarez3B,0,A. Alvarez,Armando Alvarez,Armando Alvarez,OAK
4,Z. Gelof2B,0,J. McCarthyCF-RF,1,Z. Gelof,Zack Gelof,Zack Gelof,OAK
...,...,...,...,...,...,...,...,...
12498,E. Sosa3B-SS,0,J. YoungCF,0,E. Sosa,Edmundo Sosa,Edmundo Sosa,PHI
12499,G. StubbsC,0,N. NuñezSS,0,G. Stubbs,Garrett Stubbs,Garrett Stubbs,PHI
12500,team,6,bD. BakerPH,0,None,None,NaN,NaN
12501,NaN,NaN,team,3,None,None,NaN,NaN


In [24]:
#dropping the collumns we used to match to then match the other column
merged_df = merged_df.drop(columns=['short_name', 'matched_name'])

In [25]:
#matching the batter_team_2025 with the broken names in stats_2025 
# Step 1: Safely extract short names like "P. Sandoval"
def clean_name(raw):
    if not isinstance(raw, str):
        return None

    # Normalize unicode to remove accents (e.g., Hernández → Hernandez)
    raw = unicodedata.normalize('NFKD', raw).encode('ascii', 'ignore').decode('utf-8')

    # Remove leading garbage (e.g., bT. Taylor → T. Taylor)
    raw = re.sub(r"^[^A-Z]*", "", raw)

    # Fix missing space after period (e.g., T.Taylor → T. Taylor)
    raw = re.sub(r"([A-Z])\.([A-Z])", r"\1. \2", raw)

    # Remove trailing position labels (e.g., PH-CF, 3B, DH)
    raw = re.sub(r"[A-Z]{1,3}(-[A-Z]{1,3})?$", "", raw.strip())

    # Remove anything else weird
    raw = re.sub(r"[^\w\s\.\'\-]", "", raw)

    # Match short name: first initial + last name (including compound last names)
    match = re.match(r"([A-Z]\.\s(?:[A-Z][a-z]+(?:\s|['’\-]))*[A-Z][a-z]+)", raw)
    return match.group(1).strip() if match else None

stats_2025['short_name'] = stats_2025['away_hitters'].apply(clean_name)

# Step 2: Match short name to full name in pitchers_team
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # first initial match
            return name

    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None

# Step 3: Apply the matching
full_name_list = batter_team_2025['player'].tolist()
stats_2025['matched_name'] = stats_2025['short_name'].apply(lambda x: match_full_name(x, full_name_list))

# Step 4: Merge the DataFrames on the matched name
merged_df_5 = stats_2025.merge(batter_team_2025, left_on='matched_name', right_on='player', how='left')

In [26]:
#dropping the collumns we used to match to then match the other column
merged_df_5 = merged_df_5.drop(columns=['short_name', 'matched_name'])

In [27]:
#now we are doing the same process except on the home players this time
# Step 1: Safely extract short names like "P. Sandoval"
def clean_name(raw):
    if not isinstance(raw, str):
        return None

    # Normalize unicode to remove accents (e.g., Hernández → Hernandez)
    raw = unicodedata.normalize('NFKD', raw).encode('ascii', 'ignore').decode('utf-8')

    # Remove leading garbage (e.g., bT. Taylor → T. Taylor)
    raw = re.sub(r"^[^A-Z]*", "", raw)

    # Fix missing space after period (e.g., T.Taylor → T. Taylor)
    raw = re.sub(r"([A-Z])\.([A-Z])", r"\1. \2", raw)

    # Remove trailing position labels (e.g., PH-CF, 3B, DH)
    raw = re.sub(r"[A-Z]{1,3}(-[A-Z]{1,3})?$", "", raw.strip())

    # Remove anything else weird
    raw = re.sub(r"[^\w\s\.\'\-]", "", raw)

    # Match short name: first initial + last name (including compound last names)
    match = re.match(r"([A-Z]\.\s(?:[A-Z][a-z]+(?:\s|['’\-]))*[A-Z][a-z]+)", raw)
    return match.group(1).strip() if match else None

merged_df['short_name'] = merged_df['home_hitters'].apply(clean_name)

# Step 2: Match short name to full name in pitchers_team
# Match short names to full names
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # first initial match
            return name

    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None


# Step 3: Apply the matching
full_name_list = batter_team_2024['player'].tolist()
merged_df['matched_name'] = merged_df['short_name'].apply(lambda x: match_full_name(x, full_name_list))

# Step 4: Merge the DataFrames on the matched name
merged_df_2 = merged_df.merge(batter_team_2024, left_on='matched_name', right_on='player', how='left')

In [28]:
#now we are doing the same process except on the home players this time
def clean_name(raw):
    if not isinstance(raw, str):
        return None

    # Normalize unicode to remove accents (e.g., Hernández → Hernandez)
    raw = unicodedata.normalize('NFKD', raw).encode('ascii', 'ignore').decode('utf-8')

    # Remove leading garbage (e.g., bT. Taylor → T. Taylor)
    raw = re.sub(r"^[^A-Z]*", "", raw)

    # Fix missing space after period (e.g., T.Taylor → T. Taylor)
    raw = re.sub(r"([A-Z])\.([A-Z])", r"\1. \2", raw)

    # Remove trailing position labels (e.g., PH-CF, 3B, DH)
    raw = re.sub(r"[A-Z]{1,3}(-[A-Z]{1,3})?$", "", raw.strip())

    # Remove anything else weird
    raw = re.sub(r"[^\w\s\.\'\-]", "", raw)

    # Match short name: first initial + last name (including compound last names)
    match = re.match(r"([A-Z]\.\s(?:[A-Z][a-z]+(?:\s|['’\-]))*[A-Z][a-z]+)", raw)
    return match.group(1).strip() if match else None

merged_df_5['short_name'] = merged_df_5['home_hitters'].apply(clean_name)

# Step 2: Match short name to full name in pitchers_team
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # first initial match
            return name

    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None

# Step 3: Apply the matching
full_name_list = batter_team_2025['player'].tolist()
merged_df_5['matched_name'] = merged_df_5['short_name'].apply(lambda x: match_full_name(x, full_name_list))

# Step 4: Merge the DataFrames on the matched name
merged_df_6 = merged_df_5.merge(batter_team_2025, left_on='matched_name', right_on='player', how='left')

In [29]:
merged_df_6['team_y'].unique()

array(['NYY', nan, 'WSH', 'ARI', 'MIA', 'CIN', 'CHW', 'TEX', 'SFG', 'KCR',
       'STL', 'HOU', 'TBR', 'SDP', 'SEA', 'TOR', 'LAD', 'BAL', 'CHC',
       'PHI', 'MIL', 'CLE', 'MIN', 'NYM', 'PIT', 'BOS', 'COL', 'DET',
       'ATL', 'LAA'], dtype=object)

In [30]:
#dropping a lot of uneeded columns for simplicity
merged_df_2=merged_df_2[['player_x','team_x','away_RBI','player_y','team_y','home_RBI']]

In [31]:
merged_df_7=merged_df_6[['player_x','team_x','away_RBI','player_y','team_y','home_RBI']]

In [32]:
# Rename the column
def_rank_2024.rename(columns={'Rank': 'def_rank'}, inplace=True)

In [33]:
def_rank_2025.rename(columns={'Rank': 'def_rank'}, inplace=True)

In [34]:
def_rank_2025['team'].unique()

array(['New York Mets', 'Kansas City Royals', 'Detroit Tigers',
       'San Diego Padres', 'Minnesota Twins', 'Texas Rangers',
       'San Francisco Giants', 'Houston Astros', 'Boston Red Sox',
       'New York Yankees', 'Tampa Bay Rays', 'Cincinnati Reds',
       'Philadelphia Phillies', 'Los Angeles Dodgers',
       'St. Louis Cardinals', 'Atlanta Braves', 'Seattle Mariners',
       'Chicago White Sox', 'Chicago Cubs', 'Pittsburgh Pirates',
       'Milwaukee Brewers', 'Cleveland Guardians', 'Toronto Blue Jays',
       'Arizona Diamondbacks', 'Athletics', 'Los Angeles Angels',
       'Baltimore Orioles', 'Washington Nationals', 'Miami Marlins',
       'Colorado Rockies'], dtype=object)

In [35]:
team_mapping = {
    'Milwaukee Brewers': 'MIL',
    'Minnesota Twins': 'MIN',
    'Chicago Cubs': 'CHC',
    'Colorado Rockies': 'COL',
    'Washington Nationals': 'WSH',
    'Toronto Blue Jays': 'TOR',
    'Atlanta Braves': 'ATL',
    'Baltimore Orioles': 'BAL',
    'New York Yankees': 'NYY',
    'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT',
    'Kansas City Royals': 'KCR',
    'Arizona Diamondbacks': 'ARI',
    'Boston Red Sox': 'BOS',
    'Detroit Tigers': 'DET',
    'Cleveland Guardians': 'CLE',
    'Tampa Bay Rays': 'TBR',
    'San Francisco Giants': 'SFG',
    'Los Angeles Dodgers': 'LAD',
    'Seattle Mariners': 'SEA',
    'Cincinnati Reds': 'CIN',
    'Texas Rangers': 'TEX',
    'Chicago White Sox': 'CHW',
    'Los Angeles Angels': 'LAA',
    'Miami Marlins': 'MIA',
    'New York Mets': 'NYM',
    'Athletics': 'OAK',
    'St. Louis Cardinals': 'STL',
    'San Diego Padres': 'SDP',
    'Houston Astros': 'HOU'
}

def_rank_2024['team'] = def_rank_2024['team'].map(team_mapping)
def_rank_2025['team'] = def_rank_2025['team'].map(team_mapping)


In [36]:
#making both types strings
merged_df_2.loc[:, 'team_x'] = merged_df_2['team_x'].astype(str)
def_rank_2024.loc[:, 'def_rank'] = def_rank_2024['def_rank'].astype(str)
def_rank_2025.loc[:, 'def_rank'] = def_rank_2025['def_rank'].astype(str)
merged_df_7.loc[:, 'team_x'] = merged_df_7['team_x'].astype(str)


C:\Users\Joshua\AppData\Local\Temp\ipykernel_115456\389885174.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  def_rank_2024.loc[:, 'def_rank'] = def_rank_2024['def_rank'].astype(str)
C:\Users\Joshua\AppData\Local\Temp\ipykernel_115456\389885174.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  def_rank_2025.loc[:, 'def_rank'] = def_rank_2025['def_rank'].astype(str)


In [37]:
#merge df_rank_2024 into merged_df_2 to bring in the def_rank 
merged_df_2 = merged_df_2.merge(
    def_rank_2024,
    how='left',
    left_on='team_x',
    right_on='team'
)
merged_df_2 = merged_df_2.drop(columns=['team'])

In [38]:
#merge df_rank_2025 into merged_df_7 to bring in the def_rank 
merged_df_7 = merged_df_7.merge(
    def_rank_2025,
    how='left',
    left_on='team_x',
    right_on='team'
)
merged_df_7 = merged_df_7.drop(columns=['team'])

In [39]:
#merging for home players def rank
merged_df_3 = merged_df_2.merge(
    def_rank_2024,
    how='left',
    left_on='team_y',
    right_on='team'
)
merged_df_3 = merged_df_3.drop(columns=['team'])

In [40]:
#merging for home players def rank
merged_df_8 = merged_df_7.merge(
    def_rank_2025,
    how='left',
    left_on='team_y',
    right_on='team'
)
merged_df_8 = merged_df_8.drop(columns=['team'])

In [41]:
#dropping uneeded created collumns from merging and making new df so we arent using slices of a df
df=merged_df_3[['player_x','team_x','away_RBI','def_rank_y']]
df2=merged_df_3[['player_y','team_y','home_RBI','def_rank_x']]
#2025
df3=merged_df_8[['player_x','team_x','away_RBI','def_rank_y']]
df4=merged_df_8[['player_y','team_y','home_RBI','def_rank_x']]

In [42]:
#dropping all the nan corows
df.dropna()
df2.dropna()
df3.dropna()
df4.dropna()

,player_y,team_y,home_RBI,def_rank_x
0,Austin Wells,NYY,1.0,21
1,Aaron Judge,NYY,1.0,21
2,Cody Bellinger,NYY,1.0,15
3,Paul Goldschmidt,NYY,0.0,21
4,Jazz Chisholm Jr.,NYY,0.0,21
...,...,...,...,...
4889,Jose Ramirez,CLE,0.0,13
4890,Carlos Santana,CLE,0.0,13
4891,Kyle Manzardo,CLE,0.0,13
4892,Bo Naylor,CLE,0.0,13


In [43]:
#renaming the columns of the away players 
df = df.rename(columns={
    'player_x': 'player',
    'team_x': 'team',
    'away_RBI': 'RBI',
    'def_rank_y': 'def_rank'
})

In [44]:
#renaming the columns of the away players 
df3 = df3.rename(columns={
    'player_x': 'player',
    'team_x': 'team',
    'away_RBI': 'RBI',
    'def_rank_y': 'def_rank'
})

In [45]:
#renaming the columns of the home players 
df2 = df2.rename(columns={
    'player_y': 'player',
    'team_y': 'team',
    'home_RBI': 'RBI',
    'def_rank_x': 'def_rank'
})


In [46]:
#renaming the columns of the home players 
df4 = df4.rename(columns={
    'player_y': 'player',
    'team_y': 'team',
    'home_RBI': 'RBI',
    'def_rank_x': 'def_rank'
})

In [47]:
#converting types to do math on it
# Convert 'hits' to numeric (errors='coerce' turns bad values into NaN)
df['RBI'] = pd.to_numeric(df['RBI'], errors='coerce')
df2['RBI'] = pd.to_numeric(df2['RBI'], errors='coerce')
df3['RBI'] = pd.to_numeric(df3['RBI'], errors='coerce')
df4['RBI'] = pd.to_numeric(df4['RBI'], errors='coerce')

In [48]:
#putting 1s in to indicate if they got 1 or 2 hits based off the column
df['1_RBI_count'] = (df['RBI'] > 0).astype(int)
df['2_RBI_count'] = (df['RBI'] > 1.5).astype(int)
df['3_RBI_count'] = (df['RBI'] > 2.5).astype(int)
df['4_RBI_count'] = (df['RBI'] > 3.5).astype(int)


df2['1_RBI_count'] = (df2['RBI'] > 0).astype(int)
df2['2_RBI_count'] = (df2['RBI'] > 1.5).astype(int)
df2['3_RBI_count'] = (df2['RBI'] > 2.5).astype(int)
df2['4_RBI_count'] = (df2['RBI'] > 3.5).astype(int)


#2025
df3['1_RBI_count'] = (df3['RBI'] > 0).astype(int)
df3['2_RBI_count'] = (df3['RBI'] > 1.5).astype(int)
df3['3_RBI_count'] = (df3['RBI'] > 2.5).astype(int)
df3['4_RBI_count'] = (df3['RBI'] > 3.5).astype(int)


df4['1_RBI_count'] = (df4['RBI'] > 0).astype(int)
df4['2_RBI_count'] = (df4['RBI'] > 1.5).astype(int)
df4['3_RBI_count'] = (df4['RBI'] > 2.5).astype(int)
df4['4_RBI_count'] = (df4['RBI'] > 3.5).astype(int)

In [49]:
df4.dropna()

,player,team,RBI,def_rank,1_RBI_count,2_RBI_count,3_RBI_count,4_RBI_count
0,Austin Wells,NYY,1.0,21,1,0,0,0
1,Aaron Judge,NYY,1.0,21,1,0,0,0
2,Cody Bellinger,NYY,1.0,15,1,0,0,0
3,Paul Goldschmidt,NYY,0.0,21,0,0,0,0
4,Jazz Chisholm Jr.,NYY,0.0,21,0,0,0,0
...,...,...,...,...,...,...,...,...
4889,Jose Ramirez,CLE,0.0,13,0,0,0,0
4890,Carlos Santana,CLE,0.0,13,0,0,0,0
4891,Kyle Manzardo,CLE,0.0,13,0,0,0,0
4892,Bo Naylor,CLE,0.0,13,0,0,0,0


In [50]:
df3.dropna()

,player,team,RBI,def_rank,1_RBI_count,2_RBI_count,3_RBI_count,4_RBI_count
0,Jackson Chourio,MIL,0.0,10,0,0,0,0
1,Christian Yelich,MIL,0.0,10,0,0,0,0
2,Willson Contreras,STL,0.0,10,0,0,0,0
3,Rhys Hoskins,MIL,0.0,10,0,0,0,0
4,Sal Frelick,MIL,0.0,10,0,0,0,0
...,...,...,...,...,...,...,...,...
4889,Bryce Harper,PHI,0.0,22,0,0,0,0
4890,Kyle Schwarber,PHI,3.0,22,1,1,1,0
4891,Nick Castellanos,PHI,0.0,22,0,0,0,0
4892,Max Kepler,PHI,0.0,22,0,0,0,0


In [51]:
#merging one row from df and then the next row from df2 and back and fourth
# Make sure both DataFrames have the same columns
df = df.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
df3 = df3.reset_index(drop=True)
df4 = df4.reset_index(drop=True)
# Stack them row by row
interleaved = pd.concat([df, df2]).sort_index(kind='merge').reset_index(drop=True)

In [52]:
interleaved_2 = pd.concat([df3, df4]).sort_index(kind='merge').reset_index(drop=True)

In [53]:
interleaved_2.dropna()

,player,team,RBI,def_rank,1_RBI_count,2_RBI_count,3_RBI_count,4_RBI_count
0,Jackson Chourio,MIL,0.0,10,0,0,0,0
1,Austin Wells,NYY,1.0,21,1,0,0,0
2,Christian Yelich,MIL,0.0,10,0,0,0,0
3,Aaron Judge,NYY,1.0,21,1,0,0,0
4,Willson Contreras,STL,0.0,10,0,0,0,0
...,...,...,...,...,...,...,...,...
9783,Kyle Manzardo,CLE,0.0,13,0,0,0,0
9784,Max Kepler,PHI,0.0,22,0,0,0,0
9785,Bo Naylor,CLE,0.0,13,0,0,0,0
9788,Alec Bohm,PHI,0.0,22,0,0,0,0


In [54]:
interleaved.dropna()

,player,team,RBI,def_rank,1_RBI_count,2_RBI_count,3_RBI_count,4_RBI_count
0,Shea Langeliers,OAK,0.0,27,0,0,0,0
2,Tyler Soderstrom,OAK,0.0,27,0,0,0,0
4,Daz Cameron,OAK,0.0,27,0,0,0,0
6,Armando Alvarez,OAK,0.0,27,0,0,0,0
8,Zack Gelof,OAK,0.0,27,0,0,0,0
...,...,...,...,...,...,...,...,...
24995,Drew Millas,WSH,0.0,11,0,0,0,0
24996,Edmundo Sosa,PHI,0.0,23,0,0,0,0
24997,Jacob Young,WSH,0.0,11,0,0,0,0
24998,Garrett Stubbs,PHI,0.0,23,0,0,0,0


In [55]:
#making def_rank column numeric to use to split data
interleaved['def_rank'] = pd.to_numeric(interleaved['def_rank'], errors='coerce')
interleaved_2['def_rank'] = pd.to_numeric(interleaved_2['def_rank'], errors='coerce')

In [56]:
#unders are only against top 15 teams and overs are only against bot 15 teams, so we split then to get the overs and unders df
unders = interleaved[interleaved['def_rank'] < 16].reset_index(drop=True)
overs = interleaved[interleaved['def_rank'] > 15].reset_index(drop=True)
#2025
unders_2025 = interleaved_2[interleaved_2['def_rank'] < 16].reset_index(drop=True)
overs_2025 = interleaved_2[interleaved_2['def_rank'] > 15].reset_index(drop=True)


In [57]:
#dropping nans
unders.dropna()
overs.dropna()
unders_2025.dropna()
overs_2025.dropna()

,player,team,RBI,def_rank,1_RBI_count,2_RBI_count,3_RBI_count,4_RBI_count
0,Austin Wells,NYY,1.0,21.0,1,0,0,0
1,Aaron Judge,NYY,1.0,21.0,1,0,0,0
2,Paul Goldschmidt,NYY,0.0,21.0,0,0,0,0
3,Jazz Chisholm Jr.,NYY,0.0,21.0,0,0,0,0
4,Jasson Dominguez,NYY,0.0,21.0,0,0,0,0
...,...,...,...,...,...,...,...,...
3638,Bryce Harper,PHI,0.0,22.0,0,0,0,0
3639,Kyle Schwarber,PHI,3.0,22.0,1,1,1,0
3640,Nick Castellanos,PHI,0.0,22.0,0,0,0,0
3641,Max Kepler,PHI,0.0,22.0,0,0,0,0


In [58]:
#stacking overs on top of overs_2025 to make sure they are in order to count freq
stacked_overs = pd.concat([overs, overs_2025], ignore_index=True)

In [59]:
#stacking unders on top of unders_2025 to make sure they are in order to count freq
stacked_unders = pd.concat([unders, unders_2025], ignore_index=True)

In [60]:
#making games collumn
stacked_overs['games_played'] = 1
stacked_unders['games_played'] = 1

In [61]:
stacked_overs.dropna()
stacked_unders.dropna()

,player,team,RBI,def_rank,1_RBI_count,2_RBI_count,3_RBI_count,4_RBI_count,games_played
0,Andrew McCutchen,PIT,0.0,2.0,0,0,0,0,1
1,Bryan Reynolds,PIT,0.0,2.0,0,0,0,0,1
2,Connor Joe,PIT,0.0,2.0,0,0,0,0,1
3,Oneil Cruz,PIT,0.0,2.0,0,0,0,0,1
4,Edward Olivares,PIT,1.0,2.0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...
13642,Jose Ramirez,CLE,0.0,13.0,0,0,0,0,1
13643,Carlos Santana,CLE,0.0,13.0,0,0,0,0,1
13644,Kyle Manzardo,CLE,0.0,13.0,0,0,0,0,1
13645,Bo Naylor,CLE,0.0,13.0,0,0,0,0,1


In [62]:
#making a new df that sums the 1_hit_count and 2_hit_count and games_played and then creates the season frequency for overs
overs_freq_2 = stacked_overs.groupby('player', as_index=False).agg({
    '1_RBI_count': 'sum',
    '2_RBI_count': 'sum',
    '3_RBI_count': 'sum',
    '4_RBI_count': 'sum',
    'games_played': 'sum'
})

# Add a column for hit rate aka freq
overs_freq_2['1_RBI_rate'] = overs_freq_2['1_RBI_count'] / overs_freq_2['games_played']
overs_freq_2['2_RBI_rate'] = overs_freq_2['2_RBI_count'] / overs_freq_2['games_played']
overs_freq_2['3_RBI_rate'] = overs_freq_2['3_RBI_count'] / overs_freq_2['games_played']
overs_freq_2['4_RBI_rate'] = overs_freq_2['4_RBI_count'] / overs_freq_2['games_played']

In [63]:
#dropping all players that do not have at least 15 games played in this category
overs_freq_2 = overs_freq_2[overs_freq_2['games_played'] > 14].reset_index(drop=True)

In [64]:
#making a new df that sums the 1_hit_count and 2_hit_count and games_played and then creates the season frequency for unders 
unders_freq_2 = stacked_unders.groupby('player', as_index=False).agg({
    '1_RBI_count': 'sum',
    '2_RBI_count': 'sum',
    '3_RBI_count': 'sum',
    '4_RBI_count': 'sum',
    'games_played': 'sum'
})

# Add a column for hit rate
unders_freq_2['1_RBI_rate'] = unders_freq_2['1_RBI_count'] / unders_freq_2['games_played']
unders_freq_2['2_RBI_rate'] = unders_freq_2['2_RBI_count'] / unders_freq_2['games_played']
unders_freq_2['3_RBI_rate'] = unders_freq_2['3_RBI_count'] / unders_freq_2['games_played']
unders_freq_2['4_RBI_rate'] = unders_freq_2['4_RBI_count'] / unders_freq_2['games_played']



In [65]:
#dropping all players that do not have at least 10 games played in this category
unders_freq_2 = unders_freq_2[unders_freq_2['games_played'] > 14].reset_index(drop=True)

In [66]:
#checking if distrobition is correct, merging them on name to make sure they average less hits against better pitchers and vise versa
final_freq4 = overs_freq_2.merge(
    unders_freq_2,
    on='player',
    how='inner',
    suffixes=('_over', '_under')
)

In [67]:
#dropping all rows that do not meet the creteria: avg less hits against better teams, and more against worse teams
final_freq4 = final_freq4[final_freq4['1_RBI_rate_under'] <= final_freq4['1_RBI_rate_over']].reset_index(drop=True)

In [68]:
#making the final dataframes before sending them to the final script with odds
final_over_df=final_freq4[['player','1_RBI_rate_over','2_RBI_rate_over','3_RBI_rate_over','4_RBI_rate_over']]
final_under_df=final_freq4[['player','1_RBI_rate_under','2_RBI_rate_under','3_RBI_rate_under','4_RBI_rate_under']]

In [69]:
final_over_df

,player,1_RBI_rate_over,2_RBI_rate_over,3_RBI_rate_over,4_RBI_rate_over
0,Aaron Judge,0.552632,0.263158,0.131579,0.026316
1,Adam Frazier,0.266667,0.033333,0.000000,0.000000
2,Addison Barger,0.363636,0.090909,0.000000,0.000000
3,Adley Rutschman,0.243902,0.048780,0.000000,0.000000
4,Adolis Garcia,0.348837,0.162791,0.069767,0.000000
...,...,...,...,...,...
209,Wyatt Langford,0.406250,0.218750,0.093750,0.031250
210,Xavier Edwards,0.352941,0.117647,0.000000,0.000000
211,Yandy Diaz,0.372093,0.127907,0.081395,0.023256
212,Yordan Alvarez,0.500000,0.152174,0.065217,0.021739


In [70]:
final_under_df

,player,1_RBI_rate_under,2_RBI_rate_under,3_RBI_rate_under,4_RBI_rate_under
0,Aaron Judge,0.475000,0.250000,0.175000,0.050000
1,Adam Frazier,0.125000,0.000000,0.000000,0.000000
2,Addison Barger,0.200000,0.133333,0.000000,0.000000
3,Adley Rutschman,0.171429,0.085714,0.028571,0.000000
4,Adolis Garcia,0.205882,0.029412,0.029412,0.000000
...,...,...,...,...,...
209,Wyatt Langford,0.151515,0.030303,0.030303,0.030303
210,Xavier Edwards,0.243902,0.048780,0.024390,0.000000
211,Yandy Diaz,0.287879,0.090909,0.015152,0.000000
212,Yordan Alvarez,0.352941,0.176471,0.088235,0.029412


In [71]:
final_over_df.to_csv('over_RBI_rate.csv')

In [72]:
final_under_df.to_csv('under_RBI_rate.csv')

In [73]:
print('done')

done
